In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
train_data = pd.read_csv('train.csv')
train_data.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,...,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,2012-03-01,05/2012,95,1.0,...,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,2012-01-01,03/2012,72,1.0,...,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,2012-01-01,03/2012,49,1.0,...,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2012-02-01,04/2012,46,2.0,...,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2012-02-01,04/2012,80,1.0,...,3,4,5,6,7,8,9,10,11,1


In [3]:
train_data.describe()

,loan_id,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,...,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
count,1.160580e+05,116058.000000,1.160580e+05,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,...,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000,116058.000000
mean,5.494155e+11,3.868961,2.082262e+05,292.280997,67.431939,1.593186,30.742293,769.926778,2.786288,459.611565,...,0.002180,0.003533,0.003421,0.004162,0.004825,0.005359,0.006617,0.007109,0.008065,0.005480
std,2.597560e+11,0.461020,1.146851e+05,89.762415,17.291719,0.491242,9.730798,42.210706,8.096464,381.946926,...,0.062161,0.082638,0.087553,0.100961,0.113128,0.128242,0.148430,0.162884,0.178128,0.073824
min,1.000009e+11,2.250000,1.100000e+04,60.000000,6.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.244656e+11,3.500000,1.200000e+05,180.000000,57.000000,1.000000,23.000000,751.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.486239e+11,3.875000,1.830000e+05,360.000000,72.000000,2.000000,31.000000,782.000000,0.000000,740.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.743034e+11,4.125000,2.780000e+05,360.000000,80.000000,2.000000,39.000000,800.000000,0.000000,791.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.999971e+11,6.750000,1.200000e+06,360.000000,97.000000,2.000000,64.000000,840.000000,40.000000,836.000000,...,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,1.000000


In [4]:
train_data.columns

Index(['loan_id', 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12',
       'm13'],
      dtype='object')

In [5]:
train_data['loan_purpose'].unique()

array(['C86', 'B12', 'A23'], dtype=object)

In [6]:
target = train_data['m13']
train_data = train_data.drop(labels=['m13'], axis=1)

In [7]:
def preprocess(raw_data):
    cleaned_train = raw_data.drop(labels=['loan_id','source','financial_institution','origination_date','first_payment_date'], axis=1)
    cleaned_train = pd.concat([cleaned_train, pd.get_dummies(cleaned_train['loan_purpose'], prefix='loan_purpose')], axis=1)
    cleaned_train = cleaned_train.drop(labels=['loan_purpose'], axis=1)

    # Log-transform the skewed features
    skewed = ['unpaid_principal_bal']
    features_log_transformed = pd.DataFrame(data = cleaned_train)
    features_log_transformed[skewed] = cleaned_train[skewed].apply(lambda x: np.log(x + 1))

    scaler = MinMaxScaler() # default=(0, 1)
    numerical = ['interest_rate', 'unpaid_principal_bal', 'loan_term',
                 'loan_to_value', 'debt_to_income_ratio',
                 'borrower_credit_score', 'insurance_percent',
                 'co-borrower_credit_score']

    features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
    features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

    return features_log_minmax_transform

In [8]:
cleaned_train = preprocess(train_data)

/Users/praneet/anaconda3/envs/deep_learning/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [9]:
cleaned_train.head()

,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,...,m6,m7,m8,m9,m10,m11,m12,loan_purpose_A23,loan_purpose_B12,loan_purpose_C86
0,0.444444,0.632553,1.0,0.978022,1.0,0.333333,0.826190,0.75,0.000000,0.0,...,0,1,0,0,0,0,0,0,0,1
1,0.583333,0.548121,1.0,0.725275,1.0,0.682540,0.829762,0.00,0.000000,0.0,...,0,0,0,0,0,1,0,0,1,0
2,0.222222,0.746927,0.4,0.472527,1.0,0.507937,0.928571,0.00,0.000000,0.0,...,0,0,0,0,0,0,0,0,1,0
3,0.555556,0.534366,1.0,0.439560,2.0,0.682540,0.753571,0.00,0.763158,0.0,...,0,0,0,1,1,1,1,0,1,0
4,0.555556,0.516252,1.0,0.813187,1.0,0.666667,0.810714,0.00,0.000000,0.0,...,5,6,7,8,9,10,11,0,0,1


In [10]:
cleaned_train.shape

(116058, 25)

In [11]:
classifier = DecisionTreeClassifier()

In [12]:
X = np.asarray(cleaned_train)
y = np.asarray(target)

In [13]:
classifier.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
test_data = pd.read_csv('test.csv')
test_data.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,...,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,1,Y,Browning-Hart,3.875,417000,360,01/02/12,Apr-12,75,1,...,0,0,0,0,0,0,0,0,0,0
1,2,X,OTHER,4.500,113000,360,01/02/12,Apr-12,80,2,...,0,0,0,0,0,0,0,0,0,0
2,3,Y,OTHER,4.500,72000,360,01/01/12,Mar-12,75,1,...,0,0,0,0,0,0,0,0,0,0
3,4,X,"Miller, Mcclure and Allen",4.125,123000,180,01/02/12,Apr-12,41,2,...,0,0,0,0,0,0,0,0,0,0
4,5,X,Browning-Hart,3.250,166000,180,01/02/12,Apr-12,53,2,...,0,0,0,0,0,0,0,0,0,0


In [15]:
cleaned_test = preprocess(test_data)
cleaned_test.head()

/Users/praneet/anaconda3/envs/deep_learning/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,...,m6,m7,m8,m9,m10,m11,m12,loan_purpose_A23,loan_purpose_B12,loan_purpose_C86
0,0.352941,0.835631,1.0,0.760870,1,0.351852,0.941597,0.0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
1,0.500000,0.507485,1.0,0.815217,2,0.592593,0.945173,0.0,0.942308,0,...,0,0,0,0,0,0,0,0,0,1
2,0.500000,0.394211,1.0,0.760870,1,0.611111,0.846246,0.0,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
3,0.411765,0.528796,0.4,0.391304,2,0.425926,0.951132,0.0,0.977163,0,...,0,0,0,0,0,0,0,1,0,0
4,0.205882,0.604142,0.4,0.521739,2,0.203704,0.914184,0.0,0.923077,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
X_test = np.asarray(cleaned_test)

In [17]:
y_pred = classifier.predict(X_test)

In [18]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
indices = np.array([idx for idx in range(1,len(y_pred)+1)])

In [20]:
submission_data = pd.DataFrame({'loan_id': indices, 'm13': y_pred}).set_index('loan_id')
submission_data.head()

,m13
loan_id,
1,0
2,0
3,0
4,0
5,0


In [21]:
submission_data.to_csv('min_max_submission.csv')